# Klasyfikacja tekstu za pomocą BERT i GPT2

In [1]:
from transformers import BertTokenizer, \
    BertForSequenceClassification, \
    Trainer, \
    TrainingArguments, \
    DataCollatorWithPadding, \
    pipeline
from peft import PeftModel, \
    PeftConfig, \
    LoraConfig, \
    TaskType, \
    get_peft_model
from datasets import load_dataset, concatenate_datasets, Dataset
from sklearn.metrics import accuracy_score, \
    f1_score, \
    precision_score, \
    recall_score
from sklearn.manifold import TSNE
import plotly.express as px
import numpy as np
import pandas as pd
import torch

## BertForSequenceClassification

In [2]:
dataset = load_dataset(path = 'json', data_files = '../task_1/data/full_text_classification.jsonl')
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 4441
})

In [3]:
class BertSequence():
    def __init__(self, model_name, use_lora = False):
        self.convert_dict = {
            'pozytywny wydźwięk': 0, 
            'neutralny wydźwięk': 1, 
            'negatywny wydźwięk': 2, 
            'mowa nienawiści': 3
        }
        
        self.model_name = model_name
        self.use_lora = use_lora
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self._load_model()
        self.tokenizer = self._load_tokenizer()
        
        self.dataset = None
        self.train_dataset = None
        self.val_dataset = None
        self.train_tokenized = None
        self.val_tokenized = None
        self.trainer = None
        self.sentence_embeddings = None
        self.tsne_df = None
    
    def _load_model(self):
        try:
            base_model = BertForSequenceClassification.from_pretrained(
                self.model_name, num_labels = len(self.convert_dict)
            )
            if self.use_lora:
                lora_config = LoraConfig(task_type = TaskType.SEQ_CLS, 
                                         r = 64, 
                                         lora_alpha = 1, 
                                         lora_dropout = 0.1)
                base_model = get_peft_model(model = base_model, peft_config = lora_config)
            return base_model.to(self.device)
        
        except Exception as e:
            print("Error loading model:", e)
            return None
        
    def _load_tokenizer(self):
        try:
            return BertTokenizer.from_pretrained(self.model_name)
        except Exception as e:
            print("Error loading tokenizer:", e)
            return None

    def load_data(self, dataset):
        self.dataset = dataset
        return self.dataset
    
    def _convert_labels(self, text):
        text['label'] = self.convert_dict.get(text['label'], -1)
        return text

    def split_data(self, test_size = 0.2, extra_df=None, train_reduction=1):  #, sample_size = 100
        if self.dataset is None:
            raise ValueError("Dataset is not loaded.")
        
        dataset_to_split = self.dataset['train'].train_test_split(test_size = test_size)
        self.train_dataset = dataset_to_split['train'].map(self._convert_labels)
        if train_reduction < 1:
            self.train_dataset = self.train_dataset.train_test_split(test_size=train_reduction)["test"]
        self.val_dataset = dataset_to_split['test'].map(self._convert_labels)

        if extra_df is not None:
            half_texts = [text[:len(text) // 2] for text in self.val_dataset['text']]
            df_filtered = extra_df[~extra_df['text'].apply(lambda x: any(x.startswith(half) for half in half_texts))]
            self.train_dataset = concatenate_datasets([self.train_dataset, Dataset.from_pandas(df_filtered)])
        
        return self.train_dataset, self.val_dataset
    
    def _tokenize(self, batch):
        return self.tokenizer(batch['text'], padding = 'max_length', truncation = True, max_length = 128)

    def tokenize_data(self):
        if not self.train_dataset or not self.val_dataset:
            raise ValueError("Train/validation datasets are not initialized.")
        
        self.train_tokenized = self.train_dataset.map(self._tokenize, batched = True)
        self.val_tokenized = self.val_dataset.map(self._tokenize, batched = True)
        
        self.train_tokenized.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
        self.val_tokenized.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
        
        return self.train_tokenized, self.val_tokenized

    def _compute_metrics(self, pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis = 1)
        accuracy = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average = 'weighted')
        precision = precision_score(labels, preds, average = 'weighted', zero_division = np.nan)
        recall = recall_score(labels, preds, average='weighted', zero_division = np.nan)
        return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall}
    
    def create_trainer(self, output_dir = './results'):
        if not self.train_tokenized or not self.val_tokenized:
            raise ValueError("Tokenized datasets are not initialized.")
        
        training_args = TrainingArguments(
            output_dir = output_dir,
            eval_strategy = 'epoch',
            learning_rate = 2e-3,
            per_device_train_batch_size = 4,
            per_device_eval_batch_size = 4,
            num_train_epochs = 10,
            weight_decay = 0.01,
            logging_dir = './logs',
        )
        
        self.trainer = Trainer(
            model = self.model,
            args = training_args,
            train_dataset = self.train_tokenized,
            eval_dataset = self.val_tokenized,
            data_collator = DataCollatorWithPadding(tokenizer = self.tokenizer),
            compute_metrics = self._compute_metrics,
        )
        
        return self
    
    def train(self):
        if self.trainer:
            return self.trainer.train()
        else:
            raise ValueError("Trainer is not initialized.")
    
    def evaluate(self):
        if self.trainer:
            return self.trainer.evaluate()
        else:
            raise ValueError("Trainer is not initialized.")

    def _create_embeddings(self):
        inputs = self.tokenizer(self.val_dataset['text'], 
                                padding = True, 
                                truncation = True, 
                                max_length = 128, 
                                return_tensors = 'pt').to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states = True)
            last_hidden_states = outputs.hidden_states[-1]
        self.sentence_embeddings = last_hidden_states[:, 0, :].cpu()
    
    def plot_embeddings(self):
        self._create_embeddings()

        tsne = TSNE(n_components = 2, random_state = 42, perplexity = 2)
        tsne_results = tsne.fit_transform(self.sentence_embeddings)

        self.tsne_df = pd.DataFrame(tsne_results, columns = ['x', 'y'])
        label_mapping = {v: k for k, v in self.convert_dict.items()}
        self.tsne_df['label'] = [label_mapping[label] for label in self.val_dataset['label']]
        self.tsne_df['text'] = self.val_dataset['text']

        color_map = px.colors.qualitative.Vivid[:len(self.convert_dict)]
        
        fig = px.scatter(
            self.tsne_df, x = 'x', y = 'y', color = 'label',
            title = 'Wizualizacja osadzeń SentenceBERT przy użyciu t-SNE',
            labels = {'label': 'Wydźwięk'},
            hover_name = self.tsne_df['text'],
            color_discrete_sequence = color_map
        )
        
        fig.update_traces(marker = dict(size = 10), selector = dict(mode = 'markers'))
        fig.show()

In [4]:
def run_model(model_name, dataset, use_lora = False, extra_df=None, train_reduction=1):
    model = BertSequence(model_name, use_lora)
    model.load_data(dataset)
    train, test = model.split_data(extra_df=extra_df, train_reduction=train_reduction)
    train_tokenized, test_tokenized = model.tokenize_data()
    trainer = model.create_trainer()
    trainer.train()
    # metrics = trainer.evaluate()
    # print(metrics)
    model.plot_embeddings()
    
    return model

In [ ]:
model_base = run_model('dkleczek/bert-base-polish-uncased-v1', dataset, use_lora = True)

c:\Studia\.conda\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wikto\.cache\huggingface\hub\models--dkleczek--bert-base-polish-uncased-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dkleczek/bert-base-

{'loss': 1.032, 'grad_norm': 8.555021286010742, 'learning_rate': 0.0018873873873873875, 'epoch': 0.56}


                                                  
 10%|█         | 888/8880 [01:39<12:26, 10.71it/s]

{'eval_loss': 1.2841933965682983, 'eval_accuracy': 0.6209223847019123, 'eval_f1': 0.5735525839615472, 'eval_precision': 0.6829153963692204, 'eval_recall': 0.6209223847019123, 'eval_runtime': 9.5065, 'eval_samples_per_second': 93.515, 'eval_steps_per_second': 23.458, 'epoch': 1.0}


 11%|█▏        | 1000/8880 [01:50<13:21,  9.83it/s] 

...

{'loss': 0.1889, 'grad_norm': 0.06768179684877396, 'learning_rate': 0.0001981981981981982, 'epoch': 9.01}


 96%|█████████▌| 8500/8880 [15:31<00:36, 10.43it/s]

{'loss': 0.1202, 'grad_norm': 0.5885409712791443, 'learning_rate': 8.558558558558558e-05, 'epoch': 9.57}


                                                   
100%|██████████| 8880/8880 [16:17<00:00,  9.08it/s]


{'eval_loss': 2.127368688583374, 'eval_accuracy': 0.671541057367829, 'eval_f1': 0.672943989658071, 'eval_precision': 0.6786641553476217, 'eval_recall': 0.671541057367829, 'eval_runtime': 9.3973, 'eval_samples_per_second': 94.602, 'eval_steps_per_second': 23.73, 'epoch': 10.0}
{'train_runtime': 977.5688, 'train_samples_per_second': 36.335, 'train_steps_per_second': 9.084, 'train_loss': 0.485592558147671, 'epoch': 10.0}


In [7]:
generated = pd.read_json('new_samples.jsonl', lines=True, encoding='utf-8')
generated['label'] = generated['label'].map({
            'pozytywny wydźwięk': 0, 
            'neutralny wydźwięk': 1, 
            'negatywny wydźwięk': 2, 
            'mowa nienawiści': 3
        })
generated

,text,label
0,@USER @USER Jesienią mieliśmy trzy zwycięstwa :D.,0
1,@USER myślałem że coś na meczu zrobił bo dzisi...,0
2,"@USER @USER Obejrzałam dziś setk filmów, które...",0
3,@USER patrze na Wisłe i nie mam tam terminarza...,0
4,@USER @USER @USER Ale to były Twoje ostanie sł...,0
...,...,...
1495,@USER @USER Pogonili cię z MON i nadal cwaniak...,3
1496,Gdyby @USER była zwykłą celebrytką to ten gest...,3
1497,@USER Niech się pedofilami zajmijcie!,3
1498,Craxa dziwko derby blisko.....,3


In [ ]:
model_gen = run_model('dkleczek/bert-base-polish-uncased-v1', dataset, use_lora = True, extra_df=generated)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dkleczek/bert-base-polish-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 500/17380 [00:50<28:52,  9.74it/s]

{'loss': 1.2456, 'grad_norm': 5.3903279304504395, 'learning_rate': 0.0019424626006904488, 'epoch': 0.29}


  6%|▌         | 1000/17380 [01:44<29:20,  9.30it/s]

{'loss': 1.1024, 'grad_norm': 4.744230270385742, 'learning_rate': 0.0018849252013808976, 'epoch': 0.58}


  9%|▊         | 1500/17380 [02:40<30:01,  8.81it/s]

...

{'loss': 0.1311, 'grad_norm': 0.04865933954715729, 'learning_rate': 0.00010126582278481013, 'epoch': 9.49}


 98%|█████████▊| 17000/17380 [32:00<00:42,  8.97it/s]

{'loss': 0.1606, 'grad_norm': 7.040196895599365, 'learning_rate': 4.372842347525892e-05, 'epoch': 9.78}


100%|██████████| 17380/17380 [32:54<00:00,  8.80it/s]


{'eval_loss': 1.4967893362045288, 'eval_accuracy': 0.7401574803149606, 'eval_f1': 0.7394883097888073, 'eval_precision': 0.742562656779451, 'eval_recall': 0.7401574803149606, 'eval_runtime': 10.826, 'eval_samples_per_second': 82.117, 'eval_steps_per_second': 20.599, 'epoch': 10.0}
{'train_runtime': 1974.9686, 'train_samples_per_second': 35.201, 'train_steps_per_second': 8.8, 'train_loss': 0.5280103843697744, 'epoch': 10.0}


In [23]:
model_gen = run_model('dkleczek/bert-base-polish-uncased-v1', dataset, use_lora = True, extra_df=generated)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dkleczek/bert-base-polish-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 500/15290 [00:50<24:38, 10.00it/s]

{'loss': 1.1983, 'grad_norm': 6.659948348999023, 'learning_rate': 0.001934597776324395, 'epoch': 0.33}


  7%|▋         | 1000/15290 [01:39<22:50, 10.43it/s]

{'loss': 1.0941, 'grad_norm': 5.1766157150268555, 'learning_rate': 0.00186919555264879, 'epoch': 0.65}


 10%|▉         | 1500/15290 [02:28<23:06,  9.95it/s]

...

{'loss': 0.0913, 'grad_norm': 4.580182075500488, 'learning_rate': 0.00010333551340745586, 'epoch': 9.48}


 98%|█████████▊| 15000/15290 [26:20<00:28, 10.05it/s]

{'loss': 0.0904, 'grad_norm': 1.0164124965667725, 'learning_rate': 3.793328973185088e-05, 'epoch': 9.81}


100%|██████████| 15290/15290 [26:59<00:00,  9.44it/s]


{'eval_loss': 2.2727625370025635, 'eval_accuracy': 0.6580427446569179, 'eval_f1': 0.6566947730764686, 'eval_precision': 0.6563216315224196, 'eval_recall': 0.6580427446569179, 'eval_runtime': 9.8184, 'eval_samples_per_second': 90.544, 'eval_steps_per_second': 22.712, 'epoch': 10.0}
{'train_runtime': 1619.53, 'train_samples_per_second': 37.764, 'train_steps_per_second': 9.441, 'train_loss': 0.49033701006613034, 'epoch': 10.0}


# test fragmentu danych bez i z wygenerowanymi

In [5]:
model_gen = run_model('dkleczek/bert-base-polish-uncased-v1', dataset, use_lora = True, train_reduction=0.05)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dkleczek/bert-base-polish-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3552 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

...

{'eval_loss': 1.375073790550232, 'eval_accuracy': 0.6377952755905512, 'eval_f1': 0.6222534636281133, 'eval_precision': 0.6294415962443196, 'eval_recall': 0.6377952755905512, 'eval_runtime': 9.442, 'eval_samples_per_second': 94.154, 'eval_steps_per_second': 23.618, 'epoch': 8.0}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 1.417640209197998, 'eval_accuracy': 0.625421822272216, 'eval_f1': 0.6160015325582564, 'eval_precision': 0.6229914029398372, 'eval_recall': 0.625421822272216, 'eval_runtime': 9.5094, 'eval_samples_per_second': 93.487, 'eval_steps_per_second': 23.451, 'epoch': 9.0}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 1.4519213438034058, 'eval_accuracy': 0.6287964004499438, 'eval_f1': 0.6173846348403295, 'eval_precision': 0.626586172001631, 'eval_recall': 0.6287964004499438, 'eval_runtime': 9.6147, 'eval_samples_per_second': 92.462, 'eval_steps_per_second': 23.194, 'epoch': 10.0}
{'train_runtime': 145.4076, 'train_samples_per_second': 12.241, 'train_steps_per_second': 3.095, 'train_loss': 0.496083984375, 'epoch': 10.0}


In [9]:
model_gen = run_model('dkleczek/bert-base-polish-uncased-v1', dataset, use_lora = True, extra_df=generated, train_reduction=0.05)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dkleczek/bert-base-polish-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/3420 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 2.051175117492676, 'eval_accuracy': 0.33070866141732286, 'eval_f1': 0.24367805242898508, 'eval_precision': 0.5098575936485538, 'eval_recall': 0.33070866141732286, 'eval_runtime': 10.5582, 'eval_samples_per_second': 84.2, 'eval_steps_per_second': 21.121, 'epoch': 1.0}
{'loss': 1.1864, 'grad_norm': 10.053227424621582, 'learning_rate': 0.0017076023391812865, 'epoch': 1.46}


  0%|          | 0/223 [00:00<?, ?it/s]

...

{'eval_loss': 2.899261951446533, 'eval_accuracy': 0.5016872890888638, 'eval_f1': 0.5051762553763032, 'eval_precision': 0.5463616603902283, 'eval_recall': 0.5016872890888638, 'eval_runtime': 11.1252, 'eval_samples_per_second': 79.909, 'eval_steps_per_second': 20.045, 'epoch': 8.0}
{'loss': 0.0787, 'grad_norm': 0.022471295669674873, 'learning_rate': 0.0002456140350877193, 'epoch': 8.77}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 3.1391661167144775, 'eval_accuracy': 0.48931383577052867, 'eval_f1': 0.48765796619016716, 'eval_precision': 0.5544006265796164, 'eval_recall': 0.48931383577052867, 'eval_runtime': 4.4566, 'eval_samples_per_second': 199.482, 'eval_steps_per_second': 50.039, 'epoch': 9.0}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 3.1918423175811768, 'eval_accuracy': 0.48368953880764903, 'eval_f1': 0.48248057412672435, 'eval_precision': 0.5553631139021648, 'eval_recall': 0.48368953880764903, 'eval_runtime': 10.782, 'eval_samples_per_second': 82.453, 'eval_steps_per_second': 20.683, 'epoch': 10.0}
{'train_runtime': 473.0849, 'train_samples_per_second': 28.917, 'train_steps_per_second': 7.229, 'train_loss': 0.4758354733561912, 'epoch': 10.0}
